In [193]:
from pathlib import Path
import shutil
import pandas as pd
from lxml import etree
import pickle as pk
import re
import codecs
import nltk
import json

nltk.download("punkt")

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

[nltk_data] Downloading package punkt to /home/lukel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [194]:
def copy_original_data():
    uids = []
    data = Path("../original/data/")
    papers = data.glob("**/Reference_XML/*.xml")
    summaries = data.glob("**/**/*.human.txt")

    for summary in summaries:
        uid = re.sub("([A-Z]\d{2}-\d{4})(.*)", "\g<1>", summary.stem)
        uids.append(uid)
        path = f"../data/summaries/{summary.stem.replace('.human', '')}.txt"
        shutil.copyfile(summary, path)

    for paper in papers:
        if paper.stem in uids:
            shutil.copyfile(paper, f"../data/papers/{paper.stem}.xml")

In [195]:
def parse_xml(path):
    with codecs.open(path, "r", encoding="latin-1") as file:
        try:
            xml = file.read()
        except Exception as e:
            print("Could not parse: ", paper)
            print(e)

        root = etree.fromstring(xml)
        title = root.find("./S[@sid='0']").text if not None else ""
        text = root.xpath(".//S[not(@sid = '0')]")
        sentences = [s.text for s in text if s.text is not None]
        # sids = [s.attrib.get("sid") for s in text if s.text is not None]

        return title, text, sentences

In [196]:
def splitlines_summaries(summaries):
    sentences = []

    for summary in summaries:
        with codecs.open(summary, "r+", encoding="latin-1") as s:
            text = s.read().strip()
            text = re.sub(r'([a-z])\.([A-Z])', r'\1. \2', text)

            split = nltk.tokenize.sent_tokenize(text)
            sentences.append(split)

            s.seek(0)
            s.writelines([line + "\n" for line in split])
            s.truncate()
            
    return sentences

In [197]:
def get_papers():
    papers = Path("../data/papers").glob("*.xml")
    data = []

    for path in papers:
        title, text, paper_sentences = parse_xml(path)
        uid = path.stem.replace("", "")
        summaries = Path("../data/summaries").glob(f"*{uid}*.txt")
        path_summaries = [str(s) for s in summaries]

        paper = {
            "uid": uid,
            "title": title,
            "raw_paper": paper_sentences,
            "path_paper": path,
            "path_summaries": path_summaries,
        }

        data.append(paper)

    return sorted(data, key=lambda x: x["uid"])

In [198]:
def prepare_for_annotation(summaries):
    paths = []
    for summary in summaries:
        annotations = []

        with codecs.open(summary, "r", encoding="latin-1") as file:
            for index, sentence in enumerate(file, start=1):
                annotation = {
                    "source_id": index,
                    "target_id": None,
                    "summary_text": sentence.strip(),
                    "paper_text": "",
                    "strategy": "",
                }
                annotations.append(annotation)

        path = f"../data/annotation/{summary.split('/')[-1].replace('.txt', '')}.json"
        paths.append(path)

        with open(path, "w") as f:
            json.dump(annotations, f, ensure_ascii=False, indent=4)

    return paths

In [199]:
if __name__ == "__main__":
    Path("../data/papers").mkdir(exist_ok=True, parents=True)
    Path("../data/summaries").mkdir(exist_ok=True, parents=True)
    Path("../data/annotation").mkdir(exist_ok=True, parents=True)   

    copy_original_data()
    data = get_papers()

    df = pd.DataFrame(data)
    df["raw_summaries"] = df["path_summaries"].apply(lambda s: splitlines_summaries(s))
    df["path_annotations"] = df["path_summaries"].apply(lambda s: prepare_for_annotation(s))
    
    df.to_pickle("../data/data.pkl")
    df.to_csv("../data/data.csv")

    display(df)

,uid,title,raw_paper,path_paper,path_summaries,raw_summaries,path_annotations
0,A00-2018,A Maximum-Entropy-Inspired Parser *,"[We present a new parser for parsing down to Penn tree-bank style parse trees that achieves 90.1% average precision/recall for sentences of 40 and less, and for of length 100 and less when trained and tested on the previously established [5,9,10,15,17] &quot;standard&quot; sections of the Wall Street Journal treebank., This represents a 13% decrease in error rate over the best single-parser results on this corpus [9]., The major technical innovation is the use of a &quot;maximum-entropy-inspired&quot; model for conditioning and smoothing that let us successfully to test and combine many different conditioning events., We also present some partial results showing the effects of different conditioning information, including a surprising 2% improvement due to guessing the lexical head's p...",../data/papers/A00-2018.xml,"[../data/summaries/A00-2018_sweta.txt, ../data/summaries/A00-2018_vardha.txt, ../data/summaries/A00-2018_akanksha.txt]","[[In this paper the author aims at the major technical innovation that is the use of a ""maximum-entropy-inspired"" model for conditioning and smoothing that let successfully to test and combines many different conditioning events., They also present some partial results showing the effects of different conditioning information, including a surprising 2% improvement due to guessing the lexical head's pre-terminal before guessing the lexical head., They also talk about the new parser for parsing down to Penn tree-bank style parse trees that achieves 90.1% average precision or recall for sentences of length 40 and less, and 89.5% for sentences of length 100 and less when trained and tested on the previously established ""standard"" sections of the Wall Street Journal treebank., This represe...","[../data/annotation/A00-2018_sweta.json, ../data/annotation/A00-2018_vardha.json, ../data/annotation/A00-2018_akanksha.json]"
1,A00-2030,A Novel Use of Statistical Parsing to Extract Information from Text,"[Since 1995, a few statistical parsing algorithms have demonstrated a breakthrough in parsing accuracy, as measured against the UPenn TREEBANK as a gold standard., In this paper we report adapting a lexic al ized, probabilistic context-free parser to information extraction and evaluate this new technique on MUC-7 template elements and template relations., Since 1995, a few statistical parsing algorithms (Magerman, 1995; Collins, 1996 and 1997; Charniak, 1997; Rathnaparki, 1997) demonstrated a breakthrough in parsing accuracy, as measured against the University of Pennsylvania TREEBANK as a gold standard., Yet, relatively few have embedded one of these algorithms in a task., Chiba, (1999) was able to use such a parsing algorithm to reduce perplexity with the long term goal of improved s...",../data/papers/A00-2030.xml,"[../data/summaries/A00-2030_sweta.txt, ../data/summaries/A00-2030_vardha.txt, ../data/summaries/A00-2030_aakansha.txt]","[[In this paper the author aimed at reporting, adapting a lexicalized, probabilistic context-free parser to information extraction and evaluate the new technique on MUC-7 template elements and template relations., The author is able to integrate both syntactic and semantic information into the parsing process, thus avoiding potential errors of syntax first followed by semantics., Their parsing algorithm, trained on the UPenn TREEBANK, was run on the New York Times source to create unsupervised syntactic training which was constrained to be consistent with semantic annotation., They were able to specify relatively simple guidelines that students with no training in computational linguistics could annotate., The semantic training corpus was produced by students according to a simple set ...","[../data/annotation/A00-2030_sweta.json, ../data/annotation/A00-2030_vardha.json, ../data/annotation/A00-2030_aakansha.json]"
2,A97-1014,An Annotation Scheme f